# Online Features
Feast utilizes an online store to store online features that can be fetched in real-time at inference time.  
We populated this online database when we ran _materialize_ in the `1-setup_feast.ipynb` notebook.  
In the online store, we only store the latest feature values for our registered features, which is why we won't need to specify a time when we do online retrieval here, as opposed to what we did during the offline retrieval.

In [ ]:
import feast
import pandas as pd
from datetime import datetime
import yaml

In [ ]:
with open('feature_repo/feature_store.yaml', 'r') as file:
    fs_config_yaml = yaml.safe_load(file)
fs_config = feast.repo_config.RepoConfig(**fs_config_yaml)
fs = feast.FeatureStore(config=fs_config)

Let's use the feature service here (instead of the feature view) like we would in production.  
It's better to point to a single feature definition that packages our features.

In [ ]:
feature_service = fs.get_feature_service("serving_fs")

We then get the ID for a song and ask Feast to give us the latest feature values for it, based on the features defined in the feature service.

In [ ]:
song_properties = pd.read_parquet('../99-data_prep/song_properties.parquet')
favorite_song = song_properties.loc[song_properties["name"]=="Not Like Us"]
favorite_song

In [ ]:
online_features = fs.get_online_features(
    features=feature_service,
    entity_rows=[
        {
            "spotify_id": favorite_song["spotify_id"].values[0],
        }
    ],
)

In [ ]:
features = online_features.to_dict()
features

As you can see, these are the properties we have come to know so well during this workshop, now we have a super easy way to access the latest version of them for any given song! 🎶

## Use them for inferece
To use these values for inference is nothing we haven't done before, we simply just send them as data to our served model and get a prediction back

In [ ]:
import requests

In [ ]:
deployed_model_name = "jukebox"
infer_endpoint = "<paste-the-link-here>"
infer_url = f"{infer_endpoint}/v2/models/{deployed_model_name}/infer"

In [ ]:
def rest_request(data):
    json_data = {
        "inputs": [ 
           {
                "name": name,
                "shape": [1, 1],
                "datatype": "FP32",
                "data": [float(data[name][0])]
            }
            for name in data.keys()
        ]
    }

    response = requests.post(infer_url, json=json_data, verify=False)
    response_dict = response.json()
    return response_dict['outputs'][0]['data']

In [ ]:
data = features.copy()
del data["spotify_id"]
prediction = rest_request(data)
prediction

Now that we know how to easily get the relevant features for any song just through the ID, we can use this as a pre-processing step to make sure that even if the model or feature definition changes, we will still feed the right data into the model.